# Análise do tempo de inserção da latitude e longitude em bilhetagem
Comparativo entre bases de dados SQL e NoSQL.

# 1. Apresentação dos dados utilizados
Para a análise foram utilizadas as informações tratadas disponibilizadas em arquivos CSV, usando os dados previavemente tratados.

## 1.1 GPS
Base de rastreamento dos veículos de transporte público municipal de Fortaleza.

![Dados de GPS em CSV](../../resources/imgs/dia19_gps_csv.PNG)

## 1.2 Bilhetagem
Base de bilhetagem no sistema de transporte público municipal de Fortaleza.

![Dados de Bilhetagem em CSV](../../resources/imgs/dia19_bilhetagem_csv.png)

# 2 Apresentação das databases propostas

## 2.1 GPS
As bases de GPS são usadas para inserir a latitude e a longitude em Bilhetagem.

### 2.1.1 Estruturada

![Estrutura da base de GPS em SQL](../../resources/imgs/sql_gps.PNG)

### 2.1.2 Não-Estruturada Aninhada

![Estrutura Aninhada de GPS em NoSQL](../../resources/imgs/estrutura_gps_novembro.png)

## 2.2 Bilhetagem

### 2.2.1 Estruturada
A base estruturada foi executada em um banco de dados mysql (versão 8.0.27).
Não foram utilizadas tecnicas para melhoria dessa performance nesse tipo de banco de dados.

![Estrutura da base de Bilhetagem em SQL](../../resources/imgs/sql_bilhetagem.PNG)

### 2.2.2 Não-estruturada planar
A base não-estruturada foi executada em um banco de dados MongoDB (versão 5.0.4).
Nessa modelagem, os campos dos documentos não-estruturados foram simplesmente traduzidos do banco de dados estruturado para o não-estruturado, sem nenhum tipo de otimização.

![Estrutura Planar de Bilhetagem em NoSQL](../../resources/imgs/estrutura_bilhetagem_novembro_planar.png)

### 2.2.3 Não-estruturada aninhada
A base não-estruturada foi executada em um banco de dados MongoDB (versão 5.0.4).
Nessa modelagem, os campos dos documentos não-estruturados de bilhetagem foram aninhados com os atributos de viagem.

![Estrutura Aninhada de Bilhetagem em NoSQL](../../resources/imgs/estrutura_bilhetagem_novembro_aninhada.png)

### 2.2.4 Não-estruturada aninhada com índice
A base não-estruturada foi executada em um banco de dados MongoDB (versão 5.0.4).
Nessa modelagem, os campos dos documentos não-estruturados de bilhetagem foram aninhados com os atributos de viagem.
Aqui também foi criado um índice nos atributos timestamp e código de dicionário para acelerar as consultas ao banco de validações.

![Estrutura Aninhada de Bilhetagem em NoSQL com índices](../../resources/imgs/estrutura_bilhetagem_novembro_indice.png)

# 3 Procedimentos metodológicos

## 3.1  Tratamento inicial
Configurando as bases de dados e estruturando os modelos para a análise.
Criação da base de dados inicial

In [2]:
import sys
sys.path.insert(0,'../services')

from pymysql import connect
from pymongo import MongoClient
from DatabaseSQLService import (
    criarBancosDeDados as create_database
)
import pandas as pd
import GpsToBilhetagem as gb
from GpsService import  ( 
    inserirGpsSQL as inserir_gps_sql, 
    inserirGpsMongoDB as inserir_gps_nosql, 
    gpsSelecionarIntervaloDeTempo as gps_hora
)

gps_path = '../../resources/tratados/gps/dia19_hora.csv'
bilhe_path = '../../resources/tratados/bilhetagem/dia19_hora.csv'

conexao = connect(
    host='localhost',
    user='root',
    password='123456',
    database='bigdata_tp'
)

client = MongoClient('localhost', 27017)
create_database(conexao)
database_mongo = client['bigdata_tp']

### 3.1.1 Definição do período do experimento
Definindo o tamanho da amostra com base no intervalo de tempo.

Foram adotadas informações referentes a um intervalo de quinze minutos.

**De:** 2018-11-19 (21:00:00)
**Até:** 2018-11-19 (21:15:00)

In [ ]:
gb.bilhetagem_hora('../../resources/tratados/bilhetagem/dia19.csv')
gps_hora('../../resources/tratados/gps/dia19.csv')

## 3.2 Preparação das bases
Inserção dos dados das amostras no SQL e no MongoDB

### 3.2.2 Preparação dos dados de GPS

In [ ]:
from src.services.GpsService import inserirGpsSQL

inserirGpsSQL(conexao,'../../resources/tratados/gps/GPS7Dias/gps_novembro 05.csv')

In [ ]:
inserir_gps_nosql(database_mongo,'../../resources/tratados/gps/GPS7Dias/gps_novembro 05.csv')

In [8]:
gb.inserirLatitudeELongitudeSQL(conexao, '20:00:00','20:30:00')

1.5985295999998925

In [3]:
gb.inserirLatitudeELongitudeSQL(conexao, '20:00:00','21:00:00')

2.660219199999119

In [4]:
gb.inserirLatitudeELongitudeSQL(conexao, '13:00:00','17:00:00')

4.988128800003324

In [5]:
gb.inserirLatitudeELongitudeSQL(conexao, '11:00:00','17:00:00')

7.3253430999902776

In [6]:
gb.inserirLatitudeELongitudeSQL(conexao, '6:00:00','18:00:00')

15.13636229999247

In [ ]:
gb.inserirBilhetagemSQL(conexao,'../../resources/tratados/bilhetagem/Bilhetagem4Dias/bilhetagem_05.csv')

### 3.2.3 Preparação dos dados de bilhetagem

In [ ]:
#gb.inserirBilhetagemSQL(conexao,'../../resources/tratados/bilhetagem/Bilhetagem4Dias/bilhetagem_05.csv')
gb.inserirBilhetagemPlanarMongoDB(database_mongo,'../../resources/tratados/bilhetagem/Bilhetagem4Dias/bilhetagem_05.csv')
gb.inserirBilhetagemAninhadaMongoDB(database_mongo,'../../resources/tratados/bilhetagem/Bilhetagem4Dias/bilhetagem_05.csv')
gb.inserirBilhetagemComIndiceMongoDB(database_mongo,'../../resources/tratados/bilhetagem/Bilhetagem4Dias/bilhetagem_05.csv')

## 3.3 Execução da análise
Execução do procedimento de inserção de latitude e longitude na base de bilhetagem.

In [2]:
# sql = gb(conexao,bilhe_path)
planar_lat_long_uma_hora = gb.inserirLatitudeELongitudeNaPlanar(database_mongo,20,21)
planar_lat_long_quatro_horas = gb.inserirLatitudeELongitudeNaPlanar(database_mongo,13,17)
planar_lat_long_seis_horas = gb.inserirLatitudeELongitudeNaPlanar(database_mongo,11,17)
planar_lat_long_doze_horas = gb.inserirLatitudeELongitudeNaPlanar(database_mongo,6,18)
# planar_lat_long_doze_horas = gb.inserirLatitudeELongitudeNaPlanar(database_mongo,20,21)

print(round(planar_lat_long_uma_hora,1), round(planar_lat_long_quatro_horas,1),
      round(planar_lat_long_seis_horas,1), round(planar_lat_long_doze_horas,1))
1982.5 4920.3 6828.7 13888.8


# aninhado_lat_long = gb.inserirLatitudeELongitudeNaAninhadaMongoDB(database_mongo)
# indice_lat_long =  gb.inserirLatitudeELongitudeComIndiceMongoDB(database_mongo)

1982.5 4920.3 6828.7 13888.8


In [4]:
i_aninhada_lat_long_uma_hora = gb.inserirLatitudeELongitudeComIndiceMongoDB(database_mongo,20,21)
i_aninhada_lat_long_quatro_horas = gb.inserirLatitudeELongitudeComIndiceMongoDB(database_mongo,13,17)
i_aninhada_lat_long_seis_horas = gb.inserirLatitudeELongitudeComIndiceMongoDB(database_mongo,11,17)
i_aninhada_lat_long_doze_horas = gb.inserirLatitudeELongitudeComIndiceMongoDB(database_mongo,6,18)
# aninhada_lat_long_doze_horas = gb.inserirLatitudeELongitudeNaPlanar(database_mongo,20,21)
213.3 210.7 215.1 210.4
print(round(i_aninhada_lat_long_uma_hora,1), round(i_aninhada_lat_long_quatro_horas,1),
      round(i_aninhada_lat_long_seis_horas,1), round(i_aninhada_lat_long_doze_horas,1))

213.3 210.7 215.1 210.4


In [3]:
aninhada_lat_long_uma_hora = gb.inserirLatitudeELongitudeNaAninhadaMongoDB(database_mongo,20,21)
aninhada_lat_long_quatro_horas = gb.inserirLatitudeELongitudeNaAninhadaMongoDB(database_mongo,13,17)
aninhada_lat_long_seis_horas = gb.inserirLatitudeELongitudeNaAninhadaMongoDB(database_mongo,11,17)
aninhada_lat_long_doze_horas = gb.inserirLatitudeELongitudeNaAninhadaMongoDB(database_mongo,6,18)
# aninhada_lat_long_doze_horas = gb.inserirLatitudeELongitudeNaPlanar(database_mongo,20,21)

189.1 198.6 188.2 200.4

print(round(aninhada_lat_long_uma_hora,1), round(aninhada_lat_long_quatro_horas,1),
      round(aninhada_lat_long_seis_horas,1), round(aninhada_lat_long_doze_horas,1))

189.1 198.6 188.2 200.4


## 3.4 Analise comparativa da performance
#### Máquina utilizada para análise:
Processador: i5 9400F
Memória RAM: 8GB DDR4 2400MHz
Disco Rígido: SSD NVMe n.2 240GB
Placa de Vídeo: GTX 1050 TI 4GB GDDR5

In [7]:
gps_size = 45983
b_size = 6455
df2 = pd.DataFrame([[ gps_size, b_size,round(0)],
                    [gps_size, b_size, round(0,1)],
                    [gps_size, b_size, round(32.5,1)],
                    [gps_size,b_size,round(54.6,1)]],
                   index=['SQL','Planar','Aninhado','Indice'],
                   columns=['Tamanho de GPS (registros)', 'Tamanho de Bilhetagem (registros)','Tempo de inserção (s)'])
df2

,Tamanho de GPS (registros),Tamanho de Bilhetagem (registros),Tempo de inserção (s)
SQL,45983,6455,0.0
Planar,45983,6455,0.0
Aninhado,45983,6455,32.5
Indice,45983,6455,54.6


In [6]:
import plotly.express as px

lat, long = gb.coordinates_indice(database_mongo)
df = pd.DataFrame([lat,long],index=['latitude','longitude']).transpose()

fig = px.scatter(df, x="latitude", y="longitude")
fig.show()

AttributeError: module 'GpsToBilhetagem' has no attribute 'coordinates_indice'

32.5
